In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
from typing import List
from io import StringIO
import datetime as dt
import datetime as dt
import pandas as pd
import psycopg2
import pytz
import re
import sys
import time
import warnings
from bball_utils import TEAM_CODE_DICT, generate_unique_game_id, generate_unique_player_id
from bs4 import BeautifulSoup
from io import StringIO
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
from typing import List
from urllib.request import urlopen
from scrape_player_data import clean_player_table
from scrape_all_games_data import get_all_games_table
import psycopg2

In [2]:
# Connect to postgres db
try:
    conn = psycopg2.connect(
        dbname='sports_data',
        user='postgres',
        password='postgres',
        host='localhost',
        port='5432'
    )
except:
    print("Failure to connect to database.")

query = 'SELECT * FROM basketball.all_games'

data = pd.read_sql_query(query, conn)

# Close the cursor and connection
conn.close()

C:\Users\14086\AppData\Local\Temp\ipykernel_11332\2951955221.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(query, conn)


In [15]:
from scrape_stat_sheet_data import get_games_between
game_data = get_games_between(dt.datetime(2020, 1, 1), dt.datetime(2024, 1, 1))
s = game_data[~game_data['unique_id'].isin(data['game_id'])]['date']

c:\Users\14086\Code\sports_betting\nba_data\scrape_stat_sheet_data.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_game_data = pd.read_sql_query(query, conn, params={"start": start, "end": end})


In [26]:
def group_contiguous_dates(dates):
    # Sort the dates
    sorted_dates = sorted(set(dates))
    
    # Group contiguous dates including weekends and ignoring two-day gaps
    groups = []
    group = [sorted_dates[0]]
    for i in range(1, len(sorted_dates)):
        if sorted_dates[i] - group[-1] <= dt.timedelta(days=2):
            group.append(sorted_dates[i])
        else:
            groups.append(group)
            group = [sorted_dates[i]]
    groups.append(group)

    # Format the groups
    formatted_groups = []
    for group in groups:
        if len(group) == 1:
            formatted_groups.append(group[0].strftime("%m/%d/%y"))
        else:
            formatted_groups.append(f"{group[0].strftime('%m/%d/%y')}-{group[-1].strftime('%m/%d/%y')}")

    return formatted_groups

In [1]:
from check_missing_data import get_missing_game_data, get_missing_player_data, get_missing_statsheet_data
get_missing_player_data(2020, 2022)

Missing 1 players from database from 2020 to 2022:
	Vince Carter* on ATL


In [1]:
from check_missing_data import get_missing_game_data, get_missing_player_data, get_missing_statsheet_data
get_missing_statsheet_data(2020, 2022)

Missing stat sheet data in the following days:
2020-12-23
2020-12-26
2020-12-27
2020-12-29
2020-12-31
2020-12-29
2020-12-29
2020-12-29
2020-12-29
2020-12-29
2020-12-29
2020-12-29
	And more!


c:\Users\14086\Code\sports_betting\nba_data\scrape_stat_sheet_data.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_game_data = pd.read_sql_query(query, conn, params={"start": start, "end": end})


In [7]:
import warnings
import pandas as pd
import psycopg2
try:
    conn = psycopg2.connect(
        dbname='sports_data',
        user='postgres',
        password='postgres',
        host='localhost',
        port='5432'
    )
except:
    print("Failure to connect to database.")

query = 'SELECT * FROM basketball.all_games'
with warnings.catch_warnings(action='ignore'):
    data = pd.read_sql_query(query, conn)

conn.close()

In [9]:
data.columns

Index(['date', 'start_time', 'visitor', 'visitor_points', 'home',
       'home_points', 'ot', 'attendance', 'unique_id', 'created_at'],
      dtype='object')

In [10]:
data

,date,start_time,visitor,visitor_points,home,home_points,ot,attendance,unique_id,created_at
0,2020-12-22,19:00:00,GSW,99,BRK,125,None,0.0,be073d8f99f87fc70b0c40d6b6d210d0,2024-03-24 19:57:05.676734
1,2020-12-22,22:00:00,LAC,116,LAL,109,None,0.0,c39c31f404ba02171ffcc5d1c37c25a2,2024-03-24 19:57:05.676734
2,2020-12-23,22:30:00,DAL,102,PHO,106,None,0.0,ae0b8236b1ff6986a7c197c4916e50a9,2024-04-06 16:29:00.298325
3,2020-12-23,19:00:00,NYK,107,IND,121,None,0.0,4113cb78418c6c9357c9ca2404fc6545,2024-03-24 19:57:05.676734
4,2020-12-23,19:00:00,MIA,107,ORL,113,None,3396.0,f06a34abe2510ae3c1cd5d560e4f2cee,2024-03-24 19:57:05.676734
...,...,...,...,...,...,...,...,...,...,...
4584,2020-03-10,22:30:00,LAC,131,GSW,107,None,18064.0,127aac1a71ed57f9d0535f7af19c7069,2024-04-06 16:26:48.137440
4585,2020-03-11,19:00:00,DET,106,PHI,124,None,20172.0,44ca296b0d12b12aee7050ab49764b30,2024-04-06 16:26:48.137440
4586,2020-03-11,19:30:00,NYK,136,ATL,131,OT,15393.0,db0e0f80dd359863604579d6cf9f729f,2024-04-06 16:26:48.137440
4587,2020-03-11,19:30:00,CHO,109,MIA,98,None,19600.0,398db196262de58206aa60ded720ad50,2024-04-06 16:26:48.137440


In [11]:
start = 2020
end = 2022
import pandas as pd
from scrape_player_data import get_player_id_table
import psycopg2
import warnings
import os
import contextlib

try:
    conn = psycopg2.connect(
        dbname='sports_data',
        user='postgres',
        password='postgres',
        host='localhost',
        port='5432'
    )
except:
    print("Failure to connect to database.")

query = 'SELECT * FROM basketball.all_games'
with warnings.catch_warnings(action='ignore'):
    data = pd.read_sql_query(query, conn)

conn.close()

missing_df_list = []


In [14]:
from scrape_all_games_data import get_all_games_table
months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
for year in (start, end):
    for month in months:
        try:
            with warnings.catch_warnings(action='ignore'), open(os.devnull, "w") as f, contextlib.redirect_stdout(f):
                games_table = get_all_games_table(month, year).rename(columns={'Unique_ID': 'unique_id'})
        except Exception as e:
            print(e)
            continue

        # Merge for any players that exist only in the online player table and not the database
        missing_df = games_table.merge(data.drop_duplicates(), on='unique_id', 
                        how='left', indicator=True)
        missing_df_list.append(missing_df[missing_df['_merge'] == 'left_only'])

total_missing_df = pd.concat(missing_df_list)
total_missing_df.drop_duplicates(subset=['unique_id'])

print('=' * 70)
print(f'Missing {len(total_missing_df)} games from database from {start} to {end}:')
if len(total_missing_df) > 0:
    for index, row in total_missing_df.iterrows():
        if index < 10:
            print(f'\t{row["Visitor"]} at {row["Home"]} on {row["Date"]}')
        else:
            print(f'\tAnd more!')
            break
else:
    print('\tNone!')
print('=' * 70)

'NoneType' object has no attribute 'rename'
'NoneType' object has no attribute 'rename'
'NoneType' object has no attribute 'rename'
Missing 95 games from database from 2020 to 2022:
	And more!


In [22]:
for index, row in total_missing_df.reset_index(drop=True).iterrows():
    if index < 10:
        print(f'\t{row["Visitor"]} at {row["Home"]} on {row["Date"]}')
    else:
        print(f'\tAnd more!')
        break

	DEN at PHO on 2021-10-20
	PHO at LAL on 2021-10-22
	PHO at POR on 2021-10-23
	SAC at PHO on 2021-10-27
	CLE at PHO on 2021-10-30
	NOP at PHO on 2021-11-02
	HOU at PHO on 2021-11-04
	ATL at PHO on 2021-11-06
	PHO at SAC on 2021-11-08
	POR at PHO on 2021-11-10
	And more!


In [21]:
total_missing_df.reset_index(drop=True)

,Date,Start (ET),Visitor,VisitorPTS,Home,HomePTS,OT,Attend.,unique_id,date,start_time,visitor,visitor_points,home,home_points,ot,attendance,created_at,_merge
0,2021-10-20,22:00:00,DEN,110,PHO,98,NULL,16074,15fb75aca51296e1ce9b1b46d98a3a40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
1,2021-10-22,22:00:00,PHO,115,LAL,105,NULL,18997,bce42783255e4e25cf776fb3408a064b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
2,2021-10-23,22:00:00,PHO,105,POR,134,NULL,18558,715ac12b458387250c2de58ecc312f94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
3,2021-10-27,22:00:00,SAC,110,PHO,107,NULL,14678,2c7301dc0faea99b931dd13ef19450d6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
4,2021-10-30,22:00:00,CLE,92,PHO,101,NULL,14516,f30f3a067e155695eea6baa8bb1f0f80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2022-05-06,21:30:00,PHO,94,DAL,103,NULL,20777,3234ea69786271a242deadcd2245b49d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
91,2022-05-08,15:30:00,PHO,101,DAL,111,NULL,20610,0e18cb5bc8b921dbc17a1dfd839adbad,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
92,2022-05-10,22:00:00,DAL,80,PHO,110,NULL,17071,bf593bd648c832ba13c11623129cafc6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only
93,2022-05-12,21:30:00,PHO,86,DAL,113,NULL,20777,656d4979857baf1b3ea48642b580575e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,left_only


In [4]:
def get_missing_game_data(start: int, end: int) -> None:
    '''
    Prints out missing games data between start and end years.

    Args:
        start (int): The starting year of the query.
        end (int): The ending year of the query, inclusive.
    '''
    try:
        conn = psycopg2.connect(
            dbname='sports_data',
            user='postgres',
            password='postgres',
            host='localhost',
            port='5432'
        )
    except:
        print("Failure to connect to database.")

    query = 'SELECT * FROM basketball.all_games'
    data = pd.read_sql_query(query, conn)

    conn.close()

    missing_df_list = []
    months = ['october', 'november', 'december', 'january', 'february', 'march', 'april', 'may', 'june']
    # Move print statements to a buffer
    sys.stdout = StringIO()
    for year in range(start, end+1):
        for month in months:
            games_table = get_all_games_table(month, year)

            if games_table is not None:
                games_table = games_table.rename(columns={'Unique_ID': 'unique_id'})
                # Merge for any players that exist only in the online player table and not the database
                missing_df = games_table.merge(data.drop_duplicates(), on='unique_id', 
                                how='left', indicator=True)
                missing_df_list.append(missing_df[missing_df['_merge'] == 'left_only'])
            else:
                 pass
    # Restore stdout
    sys.stdout = sys.__stdout__

    total_missing_df = pd.concat(missing_df_list)
    total_missing_df.drop_duplicates(subset=['unique_id'])

    print('=' * 70)
    print(f'Missing games from database from {start} to {end}:')
    if len(total_missing_df) > 0:
        for index, row in total_missing_df.iterrows():
            if index < 10:
                print(f'\t{row['Visitor']} at {row['Home']} on {row['Date']}')
            else:
                print(f'\tAnd more!')
                break
    else:
        print('\tNone!')
    print('=' * 70)